In [3]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Tue Oct 31 14:10:01 2017

@author: marcelo
"""

import numpy as np
import pandas as pd
import xgboost as xgb

In [7]:
x_train = pd.read_csv('./diabetic_data.csv', na_values='?')

x_train = x_train.drop(['encounter_id', 'patient_nbr'] , axis=1);

C:\Users\const\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [8]:
x_train.loc[x_train.age== '[0-10)','age'] = 0;
x_train.loc[x_train.age== '[10-20)','age'] = 10;
x_train.loc[x_train.age== '[20-30)','age'] = 20;
x_train.loc[x_train.age== '[30-40)','age'] = 30;
x_train.loc[x_train.age== '[40-50)','age'] = 40;
x_train.loc[x_train.age== '[50-60)','age'] = 50;
x_train.loc[x_train.age== '[60-70)','age'] = 60;
x_train.loc[x_train.age== '[70-80)','age'] = 70;
x_train.loc[x_train.age== '[80-90)','age'] = 80;
x_train.loc[x_train.age== '[90-100)','age'] = 90;
x_train.age = x_train.age.astype(np.int32)

In [9]:
x_train.loc[x_train.weight== '[0-25)','weight'] = 0;
x_train.loc[x_train.weight== '[25-50)','weight'] = 25;
x_train.loc[x_train.weight== '[50-75)','weight'] = 50;
x_train.loc[x_train.weight== '[75-100)','weight'] = 75;
x_train.loc[x_train.weight== '[100-125)','weight'] = 100;
x_train.loc[x_train.weight== '[125-150)','weight'] = 125;
x_train.loc[x_train.weight== '[150-175)','weight'] = 150;
x_train.loc[x_train.weight== '[175-200)','weight'] = 175;
x_train.loc[x_train.weight== '>200','weight'] = -100;
x_train.weight = x_train.weight.astype(np.float32)

In [10]:
x_train.loc[x_train.max_glu_serum== 'None','max_glu_serum'] = 0;
x_train.loc[x_train.max_glu_serum== 'Norm','max_glu_serum'] = 100;
x_train.loc[x_train.max_glu_serum== '>200','max_glu_serum'] = 200;
x_train.loc[x_train.max_glu_serum== '>300','max_glu_serum'] = 300;
x_train.max_glu_serum = x_train.max_glu_serum.astype(np.int32)

In [11]:
x_train.loc[x_train.A1Cresult== 'None','A1Cresult'] = 0;
x_train.loc[x_train.A1Cresult== 'Norm','A1Cresult'] = 5;
x_train.loc[x_train.A1Cresult== '>7','A1Cresult'] = 7;
x_train.loc[x_train.A1Cresult== '>8','A1Cresult'] = 8;
x_train.A1Cresult = x_train.A1Cresult.astype(np.int32)

In [12]:
x_train.loc[x_train.change== 'No','change'] = 0;
x_train.loc[x_train.change== 'Ch','change'] = 1;
x_train.change = x_train.change.astype(np.int8)

In [13]:
x_train.loc[x_train.diabetesMed== 'No','diabetesMed'] = 0;
x_train.loc[x_train.diabetesMed== 'Yes','diabetesMed'] = 1;
x_train.diabetesMed = x_train.diabetesMed.astype(np.int8)

In [14]:
medications = ["metformin", "repaglinide", "nateglinide", "chlorpropamide", "glimepiride", "acetohexamide", "glipizide", "glyburide", "tolbutamide", "pioglitazone", "rosiglitazone", "acarbose", "miglitol", "troglitazone", "tolazamide", "examide", "citoglipton", "insulin", "glyburide-metformin", "glipizide-metformin", "glimepiride-pioglitazone", "metformin-rosiglitazone", "metformin-pioglitazone"]

In [15]:
for med in medications:
    x_train.loc[x_train[med] == 'No', med] = -20;
    x_train.loc[x_train[med] == 'Down', med] = -10;
    x_train.loc[x_train[med] == 'Steady', med] = 0;
    x_train.loc[x_train[med] == 'Up', med] = 10;
    x_train[med] = x_train[med].astype(np.int32)

In [16]:
categoricals = ['race', 'gender', 'payer_code', 'medical_specialty','diag_1', 'diag_2', 'diag_3']

In [17]:
for c in categoricals:
    x_train[c] = pd.Categorical(x_train[c]).codes

In [18]:
x_train.loc[x_train.readmitted != 'NO','readmitted'] = 0;
x_train.loc[x_train.readmitted == 'NO','readmitted'] = 1;

In [19]:
#x_train.loc[x_train.readmitted != '<30','readmitted'] = 0;
#x_train.loc[x_train.readmitted == '<30','readmitted'] = 1;
x_train.readmitted = x_train.readmitted.astype(np.int8)

In [20]:
y_train = x_train.readmitted
x_train = x_train.drop('readmitted', axis=1)

In [21]:
n_folds = 5
early_stopping = 10
params = {'eta': 0.02, 'max_depth': 5, 'subsample': 0.7, 'colsample_bytree': 0.7, 'objective': 'binary:logistic', 'seed': 99, 'silent': 1, 'eval_metric':'auc', 'nthread':4}


In [22]:
xg_train = xgb.DMatrix(x_train, label=y_train);

cv = xgb.cv(params, xg_train, 5000, nfold=n_folds, early_stopping_rounds=early_stopping, verbose_eval=1)

[10:25:30] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-03de431ba26204c4d-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "silent" } are not used.

[10:25:30] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-03de431ba26204c4d-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "silent" } are not used.

[10:25:30] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-03de431ba26204c4d-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "silent" } are not used.

[10:25:30] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-03de431ba26204c4d-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "silent" } are not used.

[10:25:30] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-03de431ba26204c4d-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "silent" } are not used.

[0]	train-auc:0.6647

[129]	train-auc:0.70258+0.00049	test-auc:0.69408+0.00393
[130]	train-auc:0.70267+0.00049	test-auc:0.69416+0.00394
[131]	train-auc:0.70277+0.00047	test-auc:0.69420+0.00393
[132]	train-auc:0.70284+0.00047	test-auc:0.69421+0.00394
[133]	train-auc:0.70299+0.00047	test-auc:0.69429+0.00394
[134]	train-auc:0.70311+0.00045	test-auc:0.69435+0.00395
[135]	train-auc:0.70317+0.00044	test-auc:0.69438+0.00397
[136]	train-auc:0.70326+0.00041	test-auc:0.69445+0.00399
[137]	train-auc:0.70337+0.00039	test-auc:0.69449+0.00400
[138]	train-auc:0.70349+0.00040	test-auc:0.69456+0.00402
[139]	train-auc:0.70359+0.00041	test-auc:0.69462+0.00404
[140]	train-auc:0.70366+0.00039	test-auc:0.69467+0.00406
[141]	train-auc:0.70374+0.00038	test-auc:0.69472+0.00409
[142]	train-auc:0.70386+0.00041	test-auc:0.69479+0.00409
[143]	train-auc:0.70396+0.00038	test-auc:0.69487+0.00410
[144]	train-auc:0.70405+0.00036	test-auc:0.69491+0.00411
[145]	train-auc:0.70419+0.00039	test-auc:0.69498+0.00408
[146]	train-auc:0.70430+0.00040

KeyboardInterrupt: 

In [24]:
df = pd.read_csv("./diabetic_data.csv")

In [25]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
encounter_id,101766.0,1.652016e+08,1.026403e+08,12522.0,84961194.0,152388987.0,2.302709e+08,443867222.0
patient_nbr,101766.0,5.433040e+07,3.869636e+07,135.0,23413221.0,45505143.0,8.754595e+07,189502619.0
admission_type_id,101766.0,2.024006e+00,1.445403e+00,1.0,1.0,1.0,3.000000e+00,8.0
discharge_disposition_id,101766.0,3.715642e+00,5.280166e+00,1.0,1.0,1.0,4.000000e+00,28.0
admission_source_id,101766.0,5.754437e+00,4.064081e+00,1.0,1.0,7.0,7.000000e+00,25.0
time_in_hospital,101766.0,4.395987e+00,2.985108e+00,1.0,2.0,4.0,6.000000e+00,14.0
num_lab_procedures,101766.0,4.309564e+01,1.967436e+01,1.0,31.0,44.0,5.700000e+01,132.0
num_procedures,101766.0,1.339730e+00,1.705807e+00,0.0,0.0,1.0,2.000000e+00,6.0
num_medications,101766.0,1.602184e+01,8.127566e+00,1.0,10.0,15.0,2.000000e+01,81.0
number_outpatient,101766.0,3.693572e-01,1.267265e+00,0.0,0.0,0.0,0.000000e+00,42.0
